In [1]:
import pandas as pd
import datetime as dt
import json

#### Extracting Data

In [2]:
#xls = pd.ExcelFile('./data/vendas-combustiveis-m3.xlsx')
xls = pd.read_excel('./data/vendas-combustiveis-m3.xlsx', sheet_name=['DPCache_m3', 'DPCache_m3_2', 'DPCache_m3_3'], engine='openpyxl')

In [3]:
df = pd.concat(xls, ignore_index=True)
df

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
0,GASOLINA C (m3),2000,REGIÃO NORTE,RONDÔNIA,136073.253000,9563.263000,11341.229000,9369.746000,10719.983000,11165.968000,12312.451000,11220.970000,12482.281000,13591.122000,11940.570000,11547.576000,10818.094000
1,GASOLINA C (m3),2000,REGIÃO NORTE,ACRE,3358.346000,40001.853000,3065.758000,3495.290000,2946.930000,3023.920000,3206.930000,3612.580000,3264.460000,3835.740000,3676.571000,3225.610000,3289.718000
2,GASOLINA C (m3),2000,REGIÃO NORTE,AMAZONAS,20766.918000,21180.919000,242742.352000,17615.604000,20258.200000,18741.344000,19604.023000,20221.674000,20792.616000,19912.898000,21869.338000,21145.643000,20633.175000
3,GASOLINA C (m3),2000,REGIÃO NORTE,RORAIMA,3716.032000,3200.400000,3339.332000,43338.929000,3259.300000,3636.216000,3631.569000,3348.416000,3394.016000,4078.616000,3346.616000,4029.900000,4358.516000
4,GASOLINA C (m3),2000,REGIÃO NORTE,PARÁ,29755.907000,28661.951000,28145.784000,29294.796000,359575.398000,28830.479000,32297.047000,27310.979000,29396.384000,26511.009000,36553.250000,31807.840000,31009.972000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6205,GLP - Outros (m3),2020,REGIÃO SUL,RIO GRANDE DO SUL,28255.739130,NaN,NaN,NaN,217302.217391,20717.175725,20737.911232,22258.945652,18676.981884,22157.907609,25869.240942,30176.443841,28451.871377
6206,GLP - Outros (m3),2020,REGIÃO CENTRO-OESTE,MATO GROSSO DO SUL,2738.010870,3062.471014,NaN,NaN,NaN,25132.885870,2972.963768,3259.456522,3162.367754,2285.641304,2301.239130,2519.914855,2830.820652
6207,GLP - Outros (m3),2020,REGIÃO CENTRO-OESTE,MATO GROSSO,3777.313406,3614.556159,3705.135870,NaN,NaN,NaN,33101.132246,4279.393116,4232.239130,3911.268116,2858.639493,3263.902174,3458.684783
6208,GLP - Outros (m3),2020,REGIÃO CENTRO-OESTE,GOIÁS,16549.153986,18153.253623,18046.344203,17283.744565,NaN,NaN,NaN,153119.164855,17612.538043,17169.230072,18660.853261,14036.737319,15607.309783


In [ ]:
import pandas as pd

def read_excel_sheets(xls_path):
    """Read all sheets of an Excel workbook and return a single DataFrame"""
    print(f'Loading {xls_path} into pandas')
    xl = pd.ExcelFile(xls_path)
    df = pd.DataFrame()
    columns = None
    for idx, name in enumerate(xl.sheet_names[1:4]):
        print(f'Reading sheet #{idx}: {name}')
        sheet = xl.parse(name)
        if idx == 0:
            # Save column names from the first sheet to match for append
            columns = sheet.columns
        sheet.columns = columns
        # Assume index of existing data frame when appended
        df = df.append(sheet, ignore_index=True)
    return df


In [ ]:
df = read_excel_sheets('./data/vendas-combustiveis-m3.xlsx')

#### Transforming Data

In [4]:
df_month_value = pd.melt(df, id_vars=['COMBUSTÍVEL','ANO', 'ESTADO'], value_vars=['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'], var_name='MONTH', value_name='volume')

In [5]:
look_up = {'Jan':'01', 'Fev':'02', 'Mar':'03', 'Abr':'04', 'Mai':'05','Jun':'06','Jul':'07','Ago':'08','Set':'09','Out':'10','Nov':'11','Dez':'12'}
df_month_value['MONTH'] = df_month_value['MONTH'].apply(lambda x: look_up[x])

In [6]:
df_month_value['year-month'] = pd.to_datetime(df_month_value.ANO.astype(str) + '/' + df_month_value.MONTH.astype(str) + '/' + "1").dt.date
#df_month_value['year-month'] = df_month_value['year-month'].dt.strftime('%Y-%m')

In [7]:
df_month_value[['COMBUSTÍVEL', 'unit']] = pd.DataFrame(df_month_value\
                                                         .COMBUSTÍVEL\
                                                         .str\
                                                         .split('(',1)\
                                                         .tolist(), columns = ['COMBUSTÍVEL','unit'])
df_month_value['unit'] = df_month_value['unit'].str.replace(')', '')

<ipython-input-7-38f7e809df94>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_month_value['unit'] = df_month_value['unit'].str.replace(')', '')


In [8]:
final_df = df_month_value[['year-month', 'ESTADO', 'COMBUSTÍVEL', 'unit', 'volume']]
final_df.columns = ['year-month', 'uf', 'product', 'unit', 'volume']
final_df

,year-month,uf,product,unit,volume
0,2000-01-01,RONDÔNIA,GASOLINA C,m3,136073.253000
1,2000-01-01,ACRE,GASOLINA C,m3,3358.346000
2,2000-01-01,AMAZONAS,GASOLINA C,m3,20766.918000
3,2000-01-01,RORAIMA,GASOLINA C,m3,3716.032000
4,2000-01-01,PARÁ,GASOLINA C,m3,29755.907000
...,...,...,...,...,...
74515,2020-12-01,RIO GRANDE DO SUL,GLP - Outros,m3,30176.443841
74516,2020-12-01,MATO GROSSO DO SUL,GLP - Outros,m3,2519.914855
74517,2020-12-01,MATO GROSSO,GLP - Outros,m3,3263.902174
74518,2020-12-01,GOIÁS,GLP - Outros,m3,14036.737319


In [ ]:
final_df['year-month'] = final_df['year-month'].astype(str)

In [ ]:
result = final_df.to_json(orient='records')
parsed = json.loads(result)
json.dumps(parsed, indent=4)

#### Somatória

In [10]:
final_df.dtypes

year-month     object
uf             object
product        object
unit           object
volume        float64
dtype: object

In [9]:
final_df.groupby([final_df['year-month'].dt.year, final_df['product']])['volume'].agg(['sum'])

AttributeError: Can only use .dt accessor with datetimelike values

#### Teste
* Group data by year

* Sum data

In [ ]:
final_df['year-month'] = pd.to_datetime(final_df['year-month'])

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
final_df.loc[(final_df['year-month'].dt.year == 2000)].groupby(
                                                    final_df['year-month']
                                                    .dt.year)['volume'] \
                                                    .agg(['sum'])

In [ ]:
gruped = final_df.groupby(final_df['year-month'].dt.year)['volume'].agg(['sum'])

### Loading Data

In [ ]:
import psycopg2
host        = 'localhost'
port        = '5432'
dbname      = 'anp'
tablename   = 'tb_anp_fuel_sales'
password    = 'airflow'
username    = 'airflow'

In [ ]:
def conection_db(user, password, host, port, dbname):
    """
    Create the connection and the cursor to the Postgres db 
    Parameters
    user:str
    password:str
    host:str
    port:str
    dbname:str
    """
    try:
        conn = psycopg2.connect(user=user,
                                password=password,
                                host=host,
                                port=port,
                                database=dbname)
        cur = conn.cursor()
        return conn, cur

    except (Exception, psycopg2.DatabaseError) as error:
        print('Error while connecting to PostgreSQL db:', error)

In [ ]:
def creating_table(conn, cur, query):
    """
    Used to create a table in db
    Parameters:
    conn (class: psycopg2.extensions.connection): connector to db
    cur (class: psycopg2.extensions.cursor): cursor to execute SQL command
    query (str): the sql query 
    Returns:
    Nothing. Only execute the command.
    """
    try:
        cur.execute(query)
        conn.commit()
        print("Table created successfully in PostgreSQL")

    except (Exception, psycopg2.DatabaseError) as error:
        print('Error while creating PostgreSQL table:', error) 

In [ ]:
create_table_query = f"""CREATE TABLE IF NOT EXISTS {tablename}
                    (
                        year_month      DATE, 
                        uf              TEXT,
                        product         TEXT,
                        unit            TEXT,
                        volume          DOUBLE PRECISION,
                        created_at      TIMESTAMPTZ NOT NULL DEFAULT NOW()
                    )"""

In [ ]:
def create_database(conn, cur, dbname):
    try:
        sql_create_database = f'CREATE DATABASE {dbname};'
        cur.execute(sql_create_database)
        conn.commit()
        print(f'Database {dbname} created successfully in PostgreSQL')

    except (Exception, psycopg2.DatabaseError) as error:
        print(F'Error while creating PostgresSQL database {dbname}:{error}')

### Inserting Data into Table

In [ ]:
conn, cur = conection_db(username, password, host, port, dbname)
#creating_table(conn, cur, create_table_query)

In [ ]:
def insert_data_into_table(conn, cur, query, df):
    """
    Used to insert data into a table in postgresdb
    Parameters:
    conn (class: psycopg2.extensions.connection): connector to db
    cur (class: psycopg2.extensions.cursor): cursor to execute SQL command
    query (str): the sql query 
    df (list): data cleaned
    Returns:
    Nothing. Only execute the command.
    """

    try:
        cur.executemany(query, df)
        conn.commit()
        print('Data was inserted sucessfully')
        
    except (Exception, psycopg2.Error) as error:
        print('Error while inserting data:', error)

In [ ]:
insert_query = ("""
    INSERT INTO tb_anp_fuel_sales VALUES(%s,%s,%s,%s,%s)
""")

In [ ]:
final_df['year-month'] = final_df['year-month'].astype(str)

In [ ]:
list_test = final_df.values.tolist()

In [ ]:
insert_data_into_table(conn, cur, insert_query, list_test)

In [ ]:
conn.close()

In [ ]:
def connection_dbb(postgres_conn_id):
    try:
        hook = PostgresHook(postgres_conn_id=postgres_conn_id)
        conn = hook.get_conn()
        print(f'Get conn!')
        """ cur = conn.cursor()
        print('Connection and cursor was created.')
        return conn, cur """
    except (Exception, psycopg2.DatabaseError) as error:
        print('Error while connecting to PostgreSQL db:', error)